In [ ]:
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
import torchvision.transforms as tr
import numpy as np
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment as linear_assignment

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available')

cuda:0 is available


In [ ]:
train_data=torchvision.datasets.MNIST(root='/data',train=True,download=True,transform=tr.ToTensor())
test_data=torchvision.datasets.MNIST(root='./data',train=False,download=True,transform=tr.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /data/MNIST/raw/train-images-idx3-ubyte.gz to /data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /data/MNIST/raw/train-labels-idx1-ubyte.gz to /data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
#0~8번 (정상 데이터)은 normal, 9번은 abnormal(이상치)에 넣는다.
normal = []
normal_labels = []
abnormal = []
abnormal_labels = []

for i in range(60000):
  if train_data[i][1] in [0,1,2,3,4,5,6,7,8]: 
    normal.append(train_data[i][0].numpy().reshape(28,28))
    normal_labels.append(train_data[i][1])
  elif train_data[i][1] == 9:
    abnormal.append(train_data[i][0].numpy().reshape(28,28))
    abnormal_labels.append(train_data[i][1])

In [ ]:
for i in range(10000):
  if test_data[i][1] in [0,1,2,3,4,5,6,7,8]: 
    normal.append(test_data[i][0].numpy().reshape(28,28))
    normal_labels.append(test_data[i][1])
  elif test_data[i][1] == 9:
    abnormal.append(test_data[i][0].numpy().reshape(28,28))
    abnormal_labels.append(test_data[i][1])

In [ ]:
normal = np.array(normal)
normal_labels = np.array(normal_labels)
abnormal = np.array(abnormal)
abnormal_labels = np.array(abnormal_labels)

print("normal:",normal.shape)
print("abnormal:",abnormal.shape)
print(normal.shape[0]+abnormal.shape[0])

normal: (63042, 28, 28)
abnormal: (6958, 28, 28)
70000


In [ ]:
print(np.unique(normal_labels))
print(np.unique(abnormal_labels))

[0 1 2 3 4 5 6 7 8]
[9]


In [ ]:
idx=np.random.choice(63041+1,6958,replace=False) # 0이상 A 미만
trn_idx = np.delete(np.arange(63042),idx) # A 미만에서 제거 

# trn_data -> 0-8 까지의 label을 가지는 data
# tes_data -> 0-8 까지의 label을 가지는 data 반 + 9 label 을 가지는 data 반
# clu_tes_data -> 0-8 까지의 label을 가지는 data, clustering 이 잘 되었는지 확인하기 위해 사용됨
trn_data=normal[trn_idx]
trn_label = normal_labels[trn_idx]
tes_data=np.concatenate([abnormal,normal[idx]],0)
tes_label = np.concatenate([abnormal_labels, normal_labels[idx]])
clu_tes_data = normal[idx]
clu_tes_label = normal_labels[idx]

In [ ]:
#최종 training data, test data
trn_data=trn_data.reshape(-1,1,28,28)
tes_data=tes_data.reshape(-1,1,28,28)
trn_label = trn_label.reshape(-1,1)
tes_label = tes_label.reshape(-1,1)
clu_tes_data = clu_tes_data.reshape(-1,1,28,28)
clu_tes_label = clu_tes_label.reshape(-1,1)
print(trn_data.shape)
print(tes_data.shape)
print(clu_tes_data.shape) # test data 중 정상 label 데이터 6958개
print('test label:',tes_label.shape) 

(56084, 1, 28, 28)
(13916, 1, 28, 28)
(6958, 1, 28, 28)
test label: (13916, 1)


In [ ]:
np.unique(tes_label,return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 756,  911,  752,  770,  740,  714,  741,  823,  751, 6958]))

In [ ]:
class TensorData(Dataset):
  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)
    self.len = len(self.x_data)

  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]

  def __len__(self):
    return self.len

In [ ]:
train_data=TensorData(trn_data, trn_label)
test_data=TensorData(tes_data,tes_label)
clu_tes_data = TensorData(clu_tes_data, clu_tes_label)

train_loader = DataLoader(train_data,batch_size=128,shuffle=True)
test_loader = DataLoader(test_data,batch_size=128,shuffle=True)
clu_tes_loader = DataLoader(clu_tes_data, batch_size=128, shuffle = True)

In [ ]:
#dataloader확인.
image, label = iter(train_loader).next()
print(image.shape)
image2,label2=iter(test_loader).next()
print(image2.shape,label2.shape)

torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28]) torch.Size([128, 1])


### Deep k-means

In [ ]:
#batch_size=128
num_clusters = 9
latent_size=10 # 한번 6으로 해보기

In [ ]:
class Flatten(torch.nn.Module):
  def forward(self,x):
    batch_size=x.shape[0]
    return x.view(batch_size,-1)

class Deflatten(nn.Module):
  def __init__(self,k):
    super(Deflatten,self).__init__()
    self.k=k
  def forward(self,x):
    s = x.size()
    feature_size=int((s[1]//self.k)**.5)
    return x.view(s[0],self.k,feature_size,feature_size)

In [ ]:
class Encoder(nn.Module):
  def __init__(self,latent_size):
    super(Encoder,self).__init__()

    k=16
    self.encoder = nn.Sequential(
                      nn.Conv2d(1,k,3,stride=2),
                      nn.ReLU(),
                      nn.Conv2d(k, 2*k, 3, stride = 2),
                      nn.ReLU(),
                      nn.Conv2d(2*k, 4*k, 3, stride=1),
                      nn.ReLU(),
                      Flatten(),
                      nn.Linear(1024, latent_size),
                      nn.ReLU())
    
  def forward(self, x):
    return self.encoder(x)

class Decoder(nn.Module):
  def __init__(self, latent_size):
    super(Decoder,self).__init__()

    k=16
    self.decoder = nn.Sequential(
                      nn.Linear(latent_size,1024),
                      nn.ReLU(),
                      Deflatten(4*k),
                      nn.ConvTranspose2d(4*k, 2*k,3,stride=1),
                      nn.ReLU(),
                      nn.ConvTranspose2d(2*k, k, 3, stride = 2),
                      nn.ReLU(),
                      nn.ConvTranspose2d(k,1,3,stride=2,output_padding=1),
                      nn.Sigmoid())
    
  def forward(self,x):
    return self.decoder(x)
    

In [ ]:
class Kmeans(nn.Module):
  def __init__(self,num_clusters, latent_size):
    super(Kmeans,self).__init__()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.num_clusters = num_clusters
    self.centroids = nn.Parameter(torch.rand((self.num_clusters,latent_size)).to(device))  # 랜덤으로 중심들 10개, 차원도 10개, 10×10 의 크기로 생성, 값은 [0,1)

  def argminl2distance(self,a,b):
    return torch.argmin(torch.sum((a-b)**2,dim=1),dim=0) # 거리가 최소가 되는 index 반환

  # 클러스터 개수 만큼
  def forward(self,x):
    y_assign = []
    for m in range(x.size(0)):
      h = x[m].expand(self.num_clusters,-1)
      assign = self.argminl2distance(h, self.centroids)
      y_assign.append(assign.item())
    return y_assign, self.centroids[y_assign] # 가장 가까운 index, 가장 가까운 10차원의 중심

In [ ]:
def cluster_acc(y_true,y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  D = max(y_pred.max(), y_true.max()) + 1
  w = np.zeros((D,D), dtype = np.int64)
  for i in range(y_pred.size):
    w[y_pred[i],y_true[i]] +=1
  ind = linear_assignment(w.max() - w)
  return sum([w[i,j] for i,j in zip(ind[0],ind[1])]) * 1.0 / y_pred.size

In [ ]:
def evaluation(testloader, encoder, kmeans, device):
  predictions = []
  actual = []

  # testloader의 개수만큼 돌리기 때문에 list 크기는 10000개가 됨 
  with torch.no_grad():
    for images, labels in testloader:
      inputs = images.to(device)
      labels = labels.to(device)
      latent_var = encoder(inputs)
      y_pred,_ = kmeans(latent_var)

      predictions+=y_pred
      actual+=labels.cpu().tolist() # list는 + 연산자로 더해갈 수 있음

  return cluster_acc(actual,predictions)

### T1 = 100

In [ ]:
encoder = Encoder(latent_size).to(device)
decoder = Decoder(latent_size).to(device)
kmeans = Kmeans(num_clusters, latent_size).to(device)
criterion1 = torch.nn.MSELoss()
criterion2 = torch.nn.MSELoss()
optimizer = torch.optim.Adam(list(encoder.parameters())+list(decoder.parameters())+list(kmeans.parameters()), lr=1e-3)


In [ ]:
T1=100
T2=200
lam = 1e-3
ls = 0.05

models 폴더 생성하기.

In [ ]:
for ep in range(300):
  if ep <= T1:
    alpha = lam/(T2-T1)
  else:
    alpha = lam
  
  
  running_loss = 0.0
  #### 0-8 까지의 label을 가지는 data만 학습
  for images, _ in train_loader:
    inputs = images.to(device)
    optimizer.zero_grad()
    latent_var = encoder(inputs)
    _, centroids = kmeans(latent_var.detach())
    outputs = decoder(latent_var)

    l_rec = criterion1(inputs,outputs)
    l_clt = criterion2(latent_var,centroids)
    loss = l_rec + alpha*l_clt 

    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
  
  ###
  avg_loss = running_loss/len(train_loader)

  if(ep%10==0):
    ###
    testacc=evaluation(clu_tes_loader,encoder,kmeans,device)
    print('[%d] Train loss: %.4f, Test Accuracy:%.3f' %(ep,avg_loss,testacc)) # 10의 배수인 epoch마다 avg_loss 를 print

  # ls update -> 기존의 loss(hyper parameter) 보다 작게되면 그때의 신경망, kmeans의 매개변수들을 save
  # 굳이 메 epoch마다 하는 이유는? 다 끝나고 나서 저장해도 되지 않는가?
  if avg_loss < ls:
    ls=avg_loss

    torch.save(encoder.state_dict(),'./models/dkm_en.pth')
    torch.save(decoder.state_dict(),'./models/dkm_de.pth')
    torch.save(kmeans.state_dict(),'./models/dkm_clt.pth') # kmeans 매개변수 : 클러스터 개수와 중심들

[0] Train loss: 0.1044, Test Accuracy:0.123
[10] Train loss: 0.0534, Test Accuracy:0.331
[20] Train loss: 0.0505, Test Accuracy:0.419


FileNotFoundError: ignored

In [ ]:
import pandas as pd
idx = pd.DataFrame(idx)
idx.to_csv('test_index.csv')

## 신경망 + clustering 학습 완료

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#forward train data.
#훈련 데이터만을 이용해 OneclassSVM을 학습시키기 위함.

encoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(mnist)/model3/dkm_en.pth'))
decoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(mnist)/model3/dkm_de.pth'))
kmeans.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(mnist)/model3/dkm_clt.pth'))

predictions_trn = [] # 가장 가까운 index들 모아둔 것
actual_trn = [] # 실제 label 값 모아둔 것
latent_features_trn = [] # latent vector 모아둔 것 ex) [0.0, 0.9820356369018555, 0.0, 0.4858146011829376, 0.46588805317878723, 2.642456293106079, 2.1144795417785645, 0.0, 1.0553699731826782, 0.0]


with torch.no_grad():
  for images, labels in train_loader:
    inputs = images.to(device)
    labels = labels.to(device)
    latent_var = encoder(inputs)
    y_pred, _= kmeans(latent_var)

    predictions_trn+=y_pred
    latent_features_trn+=latent_var.cpu().tolist()
    actual_trn+=labels.cpu().tolist()

print(cluster_acc(actual_trn,predictions_trn))

0.8940696098709079


In [ ]:
# forward test data
# test_loader : 0~8 까지의 data 반 / 9만 있는게 반 
encoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(mnist)/model3/dkm_en.pth'))
decoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(mnist)/model3/dkm_de.pth'))
kmeans.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(mnist)/model3/dkm_clt.pth'))

test_labels = []
test_latent_vectors = [] # latent vector 가 list로 계속해서 들어간 형태

with torch.no_grad():
  for images, labels in test_loader:
    inputs = images.to(device)
    latent_var = encoder(inputs)

    test_latent_vectors+=latent_var.cpu().tolist() # test data : enoding된 10차원 data의 리스트
    test_labels+=labels.cpu().tolist()

print(type(test_latent_vectors)) 
print(len(test_labels))
print(test_latent_vectors[0])
print(test_latent_vectors[1])


<class 'list'>
13916
[0.0, 1.4431662559509277, 0.0, 1.0345962047576904, 0.9986666440963745, 1.5002806186676025, 1.2120561599731445, 0.0, 1.2488456964492798, 0.0]
[0.0, 0.9869289994239807, 0.0, 1.011060118675232, 0.8131734132766724, 1.085015892982483, 0.8248283863067627, 0.0, 1.967969536781311, 0.0]


In [ ]:
from scipy.spatial import distance
centroids=kmeans.centroids.cpu().tolist()
print(distance.euclidean(centroids[0],centroids[1]))
print(distance.euclidean(centroids[0],centroids[2]))
print(distance.euclidean(centroids[0],centroids[3]))
print(distance.euclidean(centroids[0],centroids[4]))
print(distance.euclidean(centroids[0],centroids[8]))

print()
print(distance.euclidean(centroids[1],centroids[4]))



# print(test_labels[120:130])
# print(distance.euclidean(test_latent_vectors[124],test_latent_vectors[126]))
# print(distance.euclidean(test_latent_vectors[128],test_latent_vectors[129]))


1.7544279334068424
2.170465989743986
2.025736440539997
1.9393342298633864
1.3565962751543013

1.3413229491534957


In [ ]:
# DBSCAN 
# eps 와 min_samples 2개의 하이퍼 파라미터 정해야 한다
from sklearn.cluster import DBSCAN
X = np.array(test_latent_vectors)
clustering = DBSCAN(eps=0.3, min_samples=9).fit(X)

In [ ]:
import collections
print(collections.Counter(clustering.labels_))
labels=[]
for a in test_labels:
  labels.append(a[0])
  


print(collections.Counter(labels))

Counter({0: 8726, -1: 4258, 1: 366, 2: 365, 4: 125, 3: 17, 9: 14, 8: 8, 6: 8, 5: 8, 7: 8, 10: 8, 11: 5})
Counter({9: 6958, 1: 911, 7: 823, 3: 770, 0: 756, 2: 752, 8: 751, 6: 741, 4: 740, 5: 714})


In [ ]:
print(clustering.labels_[7500:8000])
print(np.unique(clustering.labels_))

[-1  0  0  0  0  0  0 -1  0  0  4  0 -1  0  0 -1 -1  0  0  0  0 -1  0  2
 -1  0 -1  0 -1  0 -1  0 -1  0  0  0 -1  0  0 -1  0  1  1  0  0  0  4 -1
 -1  0 -1  0  2  0 -1  0  0  0  0  0 -1 -1 -1  0  1  0  0  0  0  0 -1  0
  0  0  5 -1  0  0  0 -1 -1  0  0 -1  0  0  1 -1 -1  0  0  0  0 -1  0  0
  0 -1 -1  0 -1 -1  0  0  0  0  0 -1 -1 -1  0 -1 -1  0 -1  0  0  0  0 -1
  0 -1 -1  0  0 -1  0  2 -1  0  0 -1  0  0 -1  0  0  0  0  0  0 -1  0 -1
  0  0  0  0 -1  0 -1  0  0 -1  0  0 -1  0  0  0  0 -1 -1  0  2  2  0  0
  0 -1  0  0  0  0  0  0 -1 -1 -1 -1  0  0  0  0  1  0  0  0  0  0  0 -1
  0  0  0 -1  0  0  0 -1  0 -1  0  0  0  0 -1  1  0 -1 -1  0  0  0  0 -1
  0  0  0  0  0  0  0  0  2  2  0 -1 -1 -1  0 -1 -1  0  0  0  0 -1  0  0
  0 11 -1  2  0  0  0 -1  0  0  0  0 -1 -1 -1  2  0 -1 -1  4  0 -1  0  1
  0 -1  2  0  0  0  0  0 -1 -1  0 -1  0  0  4  0  0  0  0 -1  0  0  0 -1
 -1 -1  0  0 -1  0  2 -1 -1  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0
  0 -1 -1  2  0  0  0  2  0  0  0  0  0 -1 -1 -1  4

In [ ]:
predictions_trn = np.array(predictions_trn)
actual_trn = np.array(actual_trn)
latent_features_trn = np.array(latent_features_trn)

predictions_tes = np.array(predictions_tes)
actual_tes = np.array(actual_tes)
latent_features_tes = np.array(latent_features_tes)

NameError: ignored

In [ ]:
import pandas as pd

latent_feature_trn=pd.DataFrame(latent_features_trn)
actual_trn = pd.DataFrame(actual_trn)
predictions_trn = pd.DataFrame(predictions_trn)

latent_feature_tes=pd.DataFrame(latent_features_tes)
actual_tes = pd.DataFrame(actual_tes)
predictions_tes = pd.DataFrame(predictions_tes)

In [ ]:
# csv 파일로 옮기는 과정
# latent_feature_trn.to_csv('/content/drive/MyDrive/캡스톤/latent/latent_features_trn.csv')
# actual_trn.to_csv('/content/drive/MyDrive/캡스톤/latent/actual_trn.csv')
# predictions_trn.to_csv('/content/drive/MyDrive/캡스톤/latent/predictions_trn.csv')

# latent_feature_tes.to_csv('/content/drive/MyDrive/캡스톤/latent/latent_features_tes.csv')
# actual_tes.to_csv('/content/drive/MyDrive/캡스톤/latent/actual_tes.csv')
# predictions_tes.to_csv('/content/drive/MyDrive/캡스톤/latent/predictions_tes.csv')

## One class svm ensemble

In [ ]:
latent_feature_trn = latent_feature_trn.values
predictions_trn = predictions_trn.values.reshape(-1,)
latent_feature_tes = latent_feature_tes.values
predictions_tes = predictions_tes.values.reshape(-1,)
actual_tes = actual_tes.values.reshape(-1,)

In [ ]:
y_true = np.where(actual_tes == 9,1,0) # 9일때는 1, 아닌 것은 0
np.unique(y_true, return_counts = True)


In [ ]:
np.unique(predictions_trn,return_counts=True)

In [ ]:
#OneclassSVM Ensemble
from sklearn.svm import OneClassSVM
import numpy as np
import pandas as pd

class OCSVMEnsemble():

  def __init__(self,nu=0.5):
    self.nu = nu
    #self.gamma = gamma

  def fit(self, latent, pred):
    # oneclasssvm 인스턴스가 들어갈 리스트
    self.instance = []
    self.num_cluster = len(np.unique(pred))

    # 경계가 9개 생긴거
    for clu in range(self.num_cluster):
      idx = np.where(pred == clu) # 0-9 까지의 각 그룹에 해당하는 index 도출 
      #해당 군집에 속한 data
      clu_data = latent[idx]
      ocsvm = OneClassSVM(kernel = 'rbf',gamma = 'scale', nu = self.nu).fit(clu_data) # OneClassSVM 을 통한 객체 ocsvm 생성
      self.instance.append(ocsvm)

  # 모든 OneClassSVM 인스턴스를 돌면서
  # inlier -> 0 , outlier -> 1 return 하는 함수
  def predict(self,x):
    predict = 1
    # ocsvm instance 길이만큼 -> 현재 9 (0~8 의 index)
    for md in len(self.instance):
      #print(md)
      md_pred = md.predict(x) # inliers 1, outliers -1
      #print(md_pred)
      if md_pred == 1:
        return 0
    return predict


In [ ]:
# 학습 과정
ocsvm = OCSVMEnsemble(nu=0.5) 
ocsvm.fit(latent_feature_trn, predictions_trn) # train data 에서 latent vector 들 + 가장 가까운 중심의 index

In [ ]:
# test과정에서의 latent vector 들을 넣어가면서 예측하는 과정
y_pred = []
for i in latent_feature_tes:
  #print(i)
  i=i.reshape(1,-1)
  pred=ocsvm.predict(i)
  y_pred.append(pred)

In [ ]:
np.unique(y_pred, return_counts = True)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(classification_report(y_true,y_pred))

# 최대한 경계가 서로 멀리 되도록 배치하는 것이 좋을 듯

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
X = np.array([[1, 2], [2, 2], [2, 3],[8, 7], [8, 8], [25, 80]])
clustering = DBSCAN(eps=3, min_samples=2).fit(X)
print(clustering.labels_)
print(clustering)

[ 0  0  0  1  1 -1]
DBSCAN(eps=3, min_samples=2)
